# Locations
#### Miami
Latitude: 25.80 \
Longitude: -80.28 
#### NYC
Latitude: 40.78 \
Longitude: -73.97 
#### Chicago
Latitude: 41.97 \
Longitude: -87.91 
#### Austin
Latitude: 30.20 \
Longitude: -97.67 

In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

latidudes = [25.80, 40.78, 41.97, 30.20]
longitudes = [-80.28, -73.97, -87.91,-97.67]
filenames = ["weather_mia.csv", "weather_nyc.csv", "weather_chi.csv", "weather_aus.csv"]

for i in range(len(filenames)): 
	url = "https://archive-api.open-meteo.com/v1/archive"
	params = {
		"latitude": latidudes[i],
		"longitude": longitudes[i],
		"start_date": "2024-03-02",
		"end_date": "2024-03-22",
		"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum", "snowfall_sum"],
		"temperature_unit": "fahrenheit",
		"wind_speed_unit": "mph",
		"precipitation_unit": "inch",
		"timezone": "auto"
	}
	responses = openmeteo.weather_api(url, params=params)

	# Process location. 
	response = responses[0]
	print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
	print(f"Elevation {response.Elevation()} m asl")
	print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
	print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

	# Process daily data. The order of variables needs to be the same as requested.
	daily = response.Daily()
	daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
	daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
	daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()
	daily_snowfall_sum = daily.Variables(3).ValuesAsNumpy()

	daily_data = {"DATE": pd.date_range(
		start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
		end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = daily.Interval()),
		inclusive = "left"
	)}

	daily_data["TMAX"] = daily_temperature_2m_max
	daily_data["TMIN"] = daily_temperature_2m_min
	daily_data["PRCP"] = daily_precipitation_sum
	daily_data["SNWD"] = daily_snowfall_sum

	daily_dataframe = pd.DataFrame(data = daily_data)
	print(daily_dataframe)
	daily_dataframe["DATE"] = daily_dataframe["DATE"].dt.date
	daily_dataframe.to_csv(filenames[i], index = False)

Coordinates 25.834796905517578°N -80.28353881835938°E
Elevation 1.0 m asl
Timezone b'America/New_York' b'EDT'
Timezone difference to GMT+0 -14400 s
                        DATE       TMAX       TMIN      PRCP  SNWD
0  2024-03-02 04:00:00+00:00  79.793602  68.633598  0.027559   0.0
1  2024-03-03 04:00:00+00:00  82.673599  67.643600  1.125984   0.0
2  2024-03-04 04:00:00+00:00  81.323601  64.943604  0.066929   0.0
3  2024-03-05 04:00:00+00:00  82.583603  66.923599  0.007874   0.0
4  2024-03-06 04:00:00+00:00  78.803604  66.203598  0.535433   0.0
5  2024-03-07 04:00:00+00:00  78.713600  65.753601  0.456693   0.0
6  2024-03-08 04:00:00+00:00  84.293594  67.373604  0.027559   0.0
7  2024-03-09 04:00:00+00:00  84.113602  73.493599  0.074803   0.0
8  2024-03-10 04:00:00+00:00  86.453598  70.703598  0.023622   0.0
9  2024-03-11 04:00:00+00:00  77.453598  61.883598  0.000000   0.0
10 2024-03-12 04:00:00+00:00  78.983597  62.963600  0.000000   0.0
11 2024-03-13 04:00:00+00:00  81.683601  62.9636